In [1]:

import numpy as np
import tensorflow as tf

NUMBER_OF_CLASSES=5
template_features = np.genfromtxt('../data/interim/template_matching.csv', skip_header=True, dtype=float, delimiter=',')
gabriel_features = np.genfromtxt('../data/processed/combined_features.csv', skip_header=True, dtype=float, delimiter=',')
print('labels do match?', all(template_features[:, 0] == gabriel_features[:, 0]))
y = tf.one_hot(indices=template_features[:,0], depth=NUMBER_OF_CLASSES).numpy()
X = np.hstack((template_features[:,1:], gabriel_features[:,1:]))

X = X / X.max(axis=0) # alongside rows, per column

print(X.shape, y.shape)
number_of_features = X.shape[1]


KeyboardInterrupt: 

In [ ]:
from math import e


def sigmoid(x):
    return 1/(1+e**(-x))
def sigmoid2deriv(output):
    return output*(1-output)
hidden_size=4
weights_0_1=2*np.random.random((3, hidden_size))-1
weights_1_2=2*np.random.random((hidden_size, 1))-1
streetlights = np.array([
    [1,0,1],
    [0,1,1],
    [0,0,1],
    [1,1,1],
    [1,1,1]
])
labels = np.array([[1,1,0,0,1]]).T



alpha=0.2

biases_1 = 2*np.random.random((1, hidden_size))-1
biases_2 = 2*np.random.random((1, 1))-1


for _ in range(60):
    layers_0 = streetlights
    layers_1 = sigmoid(layers_0 @ weights_0_1 + biases_1) 
    layers_2 = sigmoid(layers_1 @ weights_1_2 + biases_2)
    
    layers_2_errors = (layers_2 - labels)
    
    layers_2_delta = layers_2_errors*sigmoid2deriv(layers_2)
    layers_1_delta = layers_2_delta @ weights_1_2.T * sigmoid2deriv(layers_1) 
    
    # we can add and add them.

    
    weights_1_2_gradient = layers_1.T @ layers_2_delta
    weights_0_1_gradient = layers_0.T @ layers_1_delta

    biases_2-= alpha*np.sum(layers_2_delta)
    biases_1-= alpha*np.sum(layers_1_delta, axis=0)
    
    weights_1_2-= alpha*weights_1_2_gradient
    weights_0_1-= alpha*weights_0_1_gradient

array([[-0.00430253],
       [-0.0040815 ],
       [ 0.00100334],
       [ 0.00067144]])